<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








In [40]:
!pip3 install pydbgen

    100% |████████████████████████████████| 81kB 5.2MB/s 
    100% |████████████████████████████████| 849kB 24.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
import pydbgen
from pydbgen import pydbgen


myDB = pydbgen.pydb()

def customer_data():
  
  
  return

def payment_data():
  
  #columns = ['Bankenclearing',	'ursprüngliche Mandanten-ID']#	Dokumentennummer	Zeitstempel	Vertrag	Vorname	Name	Produkt	ZREG	Vertragsverwaltungsuser	Service	PayNet-Nummer	Dokumentenschlüssel	Anzahl Seiten
  columns = ['contract_no', 'payment_amount']

  timestamp = pd.DataFrame(pd.data_range(start='1/1/2018', end='1/08/2018'))
  payment_id = pd.DataFrame(np.arange(1,100001), columns=['payment_id'])
  payment_amount = pd.DataFrame((np.random.sample(1000001)*50000), columns=['amount'])
  payment_amount = np.round_(payment_amount, decimals = 2)
  

  return

def transaction_checker_data():
  contract_no = pd.DataFrame(np.random.randint(1000001, size=100001), columns=['contract'])
  return
  
def session_checker_data():
  return





